In [109]:
from nsepy.derivatives import get_expiry_date
from datetime import timedelta
from nsepy import get_index_pe_history
from nsepy import get_history
from nsepy import history
from datetime import date
import pandas as pd
import json

In [110]:
def getExpiryDatesList():
    
    expiryDates = []
    currentYear = todays_date = date.today().year
    currentMonth = todays_date = date.today().month
    for monthCheck in range(currentMonth,13):
        expiry = get_expiry_date(year=currentYear, month=monthCheck)
        dates = list(expiry)
        expiryDates = expiryDates + dates
    expiryDates.sort()
    datejson = json.dumps(expiryDates, indent=4, sort_keys=True, default=str)
    
    return (datejson)

In [149]:
def getOptionOI(symbol,expiryDate):

    dfPE = pd.DataFrame()    
    dfCE = pd.DataFrame()
    
    currentYear = int(date.today().year)
    currentMonth = int(date.today().month)
    currentDate = int(date.today().day)
    previous_close = round(get_history(symbol=symbol, start=date(currentYear,currentMonth,currentDate) + timedelta(days=-2), end=date(currentYear,currentMonth,currentDate))["Close"].iloc[0])
    
    for price in range(5*round(previous_close*.85/5),5*round(previous_close*1.15/5),5):
        stock_opt = get_history(symbol=symbol,
                            start=date(currentYear,currentMonth,currentDate) + timedelta(days=-60),
                            end=date(currentYear,currentMonth,currentDate),
                            option_type="PE",
                            strike_price=price,
                            expiry_date=date(int(expiryDate[0:4]),int(expiryDate[5:7]),int(expiryDate[8:10])))
        dfPE = dfPE.append(stock_opt)
    dfPE = dfPE.drop(['Option Type','Expiry','Symbol','Strike Price','Open','High','Low','Close','Last','Settle Price','Number of Contracts','Turnover','Premium Turnover','Underlying'], axis=1)
    dfPE = dfPE.groupby('Date').sum()
    dfPE = dfPE[dfPE['Open Interest'] > 0]
    dfPE = (json.loads(dfPE.to_json(orient='table')))["data"]
    
    for price in range(5*round(previous_close*.85/5),5*round(previous_close*1.15/5),5):
        stock_opt = get_history(symbol=symbol,
                            start=date(currentYear,currentMonth,currentDate) + timedelta(days=-60),
                            end=date(currentYear,currentMonth,currentDate),
                            option_type="CE",
                            strike_price=price,
                            expiry_date=date(int(expiryDate[0:4]),int(expiryDate[5:7]),int(expiryDate[8:10])))
        dfCE = dfCE.append(stock_opt)
    dfCE = dfCE.drop(['Option Type','Expiry','Symbol','Strike Price','Open','High','Low','Close','Last','Settle Price','Number of Contracts','Turnover','Premium Turnover','Underlying'], axis=1)
    dfCE = dfCE.groupby('Date').sum()
    dfCE = dfCE[dfCE['Open Interest'] > 0]
    dfCE = (json.loads(dfCE.to_json(orient='table')))["data"]
    
    return {"PE" : dfPE , 
            "CE" : dfCE}

In [150]:
print(getOptionOI("SBIN","2022-11-24"))

{'PE': [{'Date': '2022-09-05T00:00:00.000Z', 'Open Interest': 1500, 'Change in OI': 1500}, {'Date': '2022-09-06T00:00:00.000Z', 'Open Interest': 1500, 'Change in OI': 0}, {'Date': '2022-09-07T00:00:00.000Z', 'Open Interest': 1500, 'Change in OI': 0}, {'Date': '2022-09-08T00:00:00.000Z', 'Open Interest': 1500, 'Change in OI': 0}, {'Date': '2022-09-09T00:00:00.000Z', 'Open Interest': 1500, 'Change in OI': 0}, {'Date': '2022-09-12T00:00:00.000Z', 'Open Interest': 1500, 'Change in OI': 0}, {'Date': '2022-09-13T00:00:00.000Z', 'Open Interest': 1500, 'Change in OI': 0}, {'Date': '2022-09-14T00:00:00.000Z', 'Open Interest': 15000, 'Change in OI': 13500}, {'Date': '2022-09-15T00:00:00.000Z', 'Open Interest': 19500, 'Change in OI': 4500}, {'Date': '2022-09-16T00:00:00.000Z', 'Open Interest': 21000, 'Change in OI': 1500}, {'Date': '2022-09-19T00:00:00.000Z', 'Open Interest': 24000, 'Change in OI': 3000}, {'Date': '2022-09-20T00:00:00.000Z', 'Open Interest': 28500, 'Change in OI': 4500}, {'Date':